In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [16]:
# # Generate synthetic sequential data
n_samples = 1000
n_features = 3
sequence_length = 20

# Generate random features
features = np.random.rand(n_samples, sequence_length, n_features)
labels = np.random.randint(0, 2, size=n_samples)

In [17]:
# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [18]:
# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)
X_val_scaled = scaler.transform(X_val.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)
X_test_scaled = scaler.transform(X_test.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)

In [19]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [20]:
# Define LSTM model
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(LSTMModel, self).__init__()
#         self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
#         self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)  # Additional LSTM layer
#         self.fc1 = nn.Linear(hidden_size, 64)  # First dense layer
#         self.fc2 = nn.Linear(64, 32)  # Second dense layer
#         self.fc3 = nn.Sigmoid()  # Third dense layer

#     def forward(self, x):
#         out, _ = self.lstm1(x)
#         out, _ = self.lstm2(out)  # Pass through the second LSTM layer
#         out = self.fc1(out[:, -1, :])
#         out = self.fc2(out)
#         out = self.fc3(out)
#         return out


In [21]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)  # Additional LSTM layer
        self.fc1 = nn.Linear(hidden_size, 64)  # First dense layer
        self.fc2 = nn.Linear(64, 32)  # Second dense layer
        self.fc3 = nn.Linear(32, 1)  # Single output neuron
        self.fc4 = nn.Sigmoid()  # Sigmoid activation for binary classification

    def forward(self, x):
        out, (h_n, c_n) = self.lstm1(x)
        out, _ = self.lstm2(out, (h_n, c_n))  # Pass through the second LSTM layer
        out = self.fc1(out[:, -1, :])
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        return out

In [22]:
# Instantiate the enhanced model
input_size = n_features
hidden_size = 64
output_size = 1
model = LSTMModel(input_size, hidden_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
# Training loop
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.unsqueeze(1))
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor.unsqueeze(1))
        print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

Epoch [1/100], Loss: 0.2516, Val Loss: 0.2505
Epoch [2/100], Loss: 0.2508, Val Loss: 0.2501
Epoch [3/100], Loss: 0.2502, Val Loss: 0.2499
Epoch [4/100], Loss: 0.2498, Val Loss: 0.2498
Epoch [5/100], Loss: 0.2494, Val Loss: 0.2499
Epoch [6/100], Loss: 0.2493, Val Loss: 0.2501
Epoch [7/100], Loss: 0.2493, Val Loss: 0.2505
Epoch [8/100], Loss: 0.2495, Val Loss: 0.2507
Epoch [9/100], Loss: 0.2496, Val Loss: 0.2507
Epoch [10/100], Loss: 0.2496, Val Loss: 0.2506
Epoch [11/100], Loss: 0.2495, Val Loss: 0.2504
Epoch [12/100], Loss: 0.2494, Val Loss: 0.2502
Epoch [13/100], Loss: 0.2493, Val Loss: 0.2501
Epoch [14/100], Loss: 0.2493, Val Loss: 0.2500
Epoch [15/100], Loss: 0.2492, Val Loss: 0.2499
Epoch [16/100], Loss: 0.2492, Val Loss: 0.2498
Epoch [17/100], Loss: 0.2492, Val Loss: 0.2498
Epoch [18/100], Loss: 0.2492, Val Loss: 0.2497
Epoch [19/100], Loss: 0.2492, Val Loss: 0.2497
Epoch [20/100], Loss: 0.2492, Val Loss: 0.2497
Epoch [21/100], Loss: 0.2492, Val Loss: 0.2497
Epoch [22/100], Loss: 

In [24]:
# Evaluate on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)

    test_loss = criterion(test_outputs, y_test_tensor.unsqueeze(1))
    print(f"Test Loss: {test_loss.item():.4f}")

Test Loss: 0.2938


In [25]:
# Save the trained LSTM model
torch.save(model.state_dict(), 'model.pt')

In [26]:
# Create an instance of your LSTM model
model = LSTMModel(input_size, hidden_size)

# Load the saved weights
model.load_state_dict(torch.load('model.pt'))
model.eval()  # Set the model to evaluation mode

LSTMModel(
  (lstm1): LSTM(3, 64, batch_first=True)
  (lstm2): LSTM(64, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (fc4): Sigmoid()
)

---------------------**Activate API Endpoint** before proceeding-------------------------

In [27]:
import requests


In [28]:
for i in range(int(n_samples*0.15)):
    input_data = {
    "input":  X_test_scaled.tolist()[i] 
}
    response = requests.post('http://localhost:5000/predict', json=input_data).json()

    threshold = 0.5
    RBP = response["output"]

    actualRBP = y_test.tolist()[i]
    print("\n", i, "RBP =", RBP, "| actual RBP =", actualRBP, end=" ")
    # if RBP >= threshold:
    #     print("| WARNING! Impending robbery.", end="")
    if ((RBP >= threshold) & (actualRBP == 1)) | ((RBP < threshold) & (actualRBP == 0)):
        print("| CORRECT", end="")



 0 RBP = 0.06298338621854782 | actual RBP = 1 
 1 RBP = 0.5009506344795227 | actual RBP = 0 
 2 RBP = 0.7553962469100952 | actual RBP = 0 
 3 RBP = 0.4552338123321533 | actual RBP = 0 | CORRECT
 4 RBP = 0.4894294738769531 | actual RBP = 0 | CORRECT
 5 RBP = 0.6012647151947021 | actual RBP = 1 | CORRECT
 6 RBP = 0.6738506555557251 | actual RBP = 0 
 7 RBP = 0.183218851685524 | actual RBP = 1 
 8 RBP = 0.6548489332199097 | actual RBP = 0 
 9 RBP = 0.4490438997745514 | actual RBP = 1 
 10 RBP = 0.5531648993492126 | actual RBP = 1 | CORRECT
 11 RBP = 0.11002562195062637 | actual RBP = 1 
 12 RBP = 0.2072993814945221 | actual RBP = 1 
 13 RBP = 0.5192915797233582 | actual RBP = 1 | CORRECT
 14 RBP = 0.6029329299926758 | actual RBP = 0 
 15 RBP = 0.5442562699317932 | actual RBP = 0 
 16 RBP = 0.5070193409919739 | actual RBP = 0 
 17 RBP = 0.46793845295906067 | actual RBP = 1 
 18 RBP = 0.5538752675056458 | actual RBP = 1 | CORRECT
 19 RBP = 0.6122048497200012 | actual RBP = 1 | CORRECT
 20 

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002DD70C9E7D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
# study first doen lstm
# work on dynamic input
# try solar dataset